In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [2]:
#para saber el punto inicial y final de la linea
line_438_busstops = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Paradas/gtfs_linha438-paradas.csv')

departure_points = line_438_busstops[(line_438_busstops["sequencia"]==1)]
departure_points


,linha,descricao,agencia,sequencia,latitude,longitude
94,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9882,-43.2283
127,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9163,-43.2512


In [3]:
#Estableciendo la conexion con la base de datos aniv_cidade_event en MongoDB
db = MongoClient().aniv_cidade_event

In [4]:
#Funcion para determinar el punto inicial y final de un recorrido de una linea de omnibus
def bus_line_initial_final_points(bus_line):
    busstops_firstpoints=[]
    for row in db.busstops.find({ "linha": bus_line,"sequencia": 1 }):
        busstops_firstpoints.insert(0, [row[u'linha'], row[u'descricao'], row[u'latitude'], row[u'longitude']])

    #convertir la lista en un dataframe de pandas
    df_departure_points = pd.DataFrame(busstops_firstpoints, columns=['linha','descricao', 'latitude', 'longitude'])
    #df_departure_points
    initial_point = df_departure_points['latitude'][0], df_departure_points['longitude'][0]
    inverted_initial_point = df_departure_points['longitude'][0], df_departure_points['latitude'][0]
    final_point = df_departure_points['latitude'][1], df_departure_points['longitude'][1]
    inverted_final_point = df_departure_points['longitude'][1], df_departure_points['latitude'][1]
    return initial_point, final_point, inverted_initial_point, inverted_final_point

In [5]:
#Llamada a la funcion bus_line_initial_final_points de la 438
initial_final_points = bus_line_initial_final_points(438)
initial_final_points

((-22.9163, -43.251199999999997),
 (-22.988199999999999, -43.228299999999997),
 (-43.251199999999997, -22.9163),
 (-43.228299999999997, -22.988199999999999))

In [6]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 1
initial_point = initial_final_points[1]
initial_point

(-22.988199999999999, -43.228299999999997)

In [7]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 0
final_point = initial_final_points[0]
final_point

(-22.9163, -43.251199999999997)

In [8]:
inverted_initial_point = initial_final_points[3]
inverted_initial_point

(-43.228299999999997, -22.988199999999999)

In [9]:
inverted_final_point = initial_final_points[2]
inverted_final_point

(-43.251199999999997, -22.9163)

In [10]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150208_line_438.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.00052543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points


In [11]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto initial_point(-22.988199999999999, -43.228299999999997) aqui se pasan a la inversa
sorted_near_initial_points = near_points_to_origin ("A27629", (-43.228299999999997, -22.988199999999999))
sorted_near_initial_points


,timestamp,latitude,longitude,speed
0,08-02-2015 19:23:27,-22.988079,-43.228588,0


In [12]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto final_point (-22.9163, -43.251199999999997). Recordar que aqui las coordenadas del punto se pasan a la inversa(long, lat)
sorted_near_final_points = near_points_to_origin ("A27629", (-43.251199999999997, -22.9163))
sorted_near_final_points

,timestamp,latitude,longitude,speed
0,08-02-2015 07:09:43,-22.916161,-43.250912,0
1,08-02-2015 07:17:41,-22.916189,-43.251049,0
2,08-02-2015 07:18:41,-22.916189,-43.251049,0
3,08-02-2015 07:19:41,-22.916189,-43.251060,0
4,08-02-2015 07:20:11,-22.916189,-43.251049,0
5,08-02-2015 07:30:01,-22.916210,-43.251080,0
6,08-02-2015 07:30:31,-22.916210,-43.251080,0
7,08-02-2015 07:31:31,-22.916370,-43.251259,11
8,08-02-2015 15:09:30,-22.916189,-43.250992,0
9,08-02-2015 15:10:30,-22.916189,-43.250999,0


In [13]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [55]:
#pintando los puntos más cerca del punto inicial
fmap_A27629 = folium.Map(location=[-22.950635, -43.210436], zoom_start=12)


#Pintando el shape de la linea
onibus438_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha438-shapes.csv')
fmap_A27629 = folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations438 = []
for row in onibus438_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations438.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])
fmap_A27629.line(locations438)


#Pintando Circulos de radio de 1km alrededor de los puntos incial y final 
fmap_A27629.simple_marker(final_point, popup='Parada de Vila Isabel')
fmap_A27629.simple_marker(initial_point, popup='Parada Leblon')

#Punto inicial
fmap_A27629.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap_A27629.circle_marker(location=initial_point, radius=50,line_color='red',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap_A27629)

In [49]:
#Pintando todas las obsercaviones del GPS para ese ese de ese bus_id
#for doc in db.dados20150208_line_438.find({"bus_id": 'A27629'}):  
    #fmap_A27629.polygon_marker( [ doc['latitude'], doc['longitude'] ], fill_color='green', num_sides=4, radius=6)
    
#embed_map(fmap_A27629)

In [56]:
#Pintando los near y los final points
for row in sorted_near_initial_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap_A27629.polygon_marker( latlon, fill_color='black', num_sides=4, radius=6)
    
for row in sorted_near_final_points.iterrows():
#The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap_A27629.polygon_marker( latlon, fill_color='black', num_sides=4, radius=6)



embed_map(fmap_A27629)

In [21]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']
initial_points_timestamp


0    08-02-2015 19:23:27
Name: timestamp, dtype: object

In [22]:
final_points_timestamp = sorted_near_final_points['timestamp']
final_points_timestamp

0     08-02-2015 07:09:43
1     08-02-2015 07:17:41
2     08-02-2015 07:18:41
3     08-02-2015 07:19:41
4     08-02-2015 07:20:11
5     08-02-2015 07:30:01
6     08-02-2015 07:30:31
7     08-02-2015 07:31:31
8     08-02-2015 15:09:30
9     08-02-2015 15:10:30
10    08-02-2015 15:11:30
11    08-02-2015 15:12:30
12    08-02-2015 15:13:30
13    08-02-2015 15:14:30
14    08-02-2015 15:15:30
15    08-02-2015 15:17:00
16    08-02-2015 15:20:49
17    08-02-2015 17:50:49
18    08-02-2015 17:51:49
19    08-02-2015 17:52:49
20    08-02-2015 17:53:49
21    08-02-2015 17:54:19
22    08-02-2015 18:01:32
23    08-02-2015 18:03:32
24    08-02-2015 18:04:32
25    08-02-2015 18:05:32
26    08-02-2015 18:07:02
27    08-02-2015 18:08:02
28    08-02-2015 18:20:57
29    08-02-2015 18:21:57
30    08-02-2015 18:22:27
31    08-02-2015 18:23:57
32    08-02-2015 20:28:57
33    08-02-2015 20:29:27
34    08-02-2015 20:30:27
35    08-02-2015 20:31:27
Name: timestamp, dtype: object

In [24]:
#Funcion que retorna el arreglo con las horas de salida del omnibus al punto inicial/final del trayecto
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif len(initial_points_timestamp_list) == 1:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours = []
    return list_departure_hours


In [25]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
leblon_departure_hours = bus_departure_hour(initial_points_timestamp)
leblon_departure_hours

[datetime.datetime(2015, 2, 8, 19, 23, 27)]

In [26]:
#Llamada a la funcion bus_departure_hour para el punto Vila Isabel  (devuelve todas las horas en que salio el omnibus de Vila Isabel)
vilaIsabel_departure_hours = bus_departure_hour(final_points_timestamp)
vilaIsabel_departure_hours

[datetime.datetime(2015, 2, 8, 7, 31, 31),
 datetime.datetime(2015, 2, 8, 15, 20, 49),
 datetime.datetime(2015, 2, 8, 18, 23, 57),
 datetime.datetime(2015, 2, 8, 20, 31, 27)]

In [27]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    if len(initial_points_timestamp_list) >1:
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif (len(initial_points_timestamp_list) == 1):
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_arrive_hours = []        
    return list_arrive_hours


In [28]:
#Llamada a la funcion bus_arrive_hour para el punto Leblon  (devuelve todas las horas en que llegó el omnibus a Leblon)
arrive_hour_to_leblon = bus_arrive_hour(initial_points_timestamp)
arrive_hour_to_leblon

[datetime.datetime(2015, 2, 8, 19, 23, 27)]

In [29]:
#Llamada a la funcion bus_arrive_hour para el punto Vila Isabel  (devuelve todas las horas en que llegó el omnibus a Vila Isabel)
arrive_hour_to_vilaisabel = bus_arrive_hour(final_points_timestamp)
arrive_hour_to_vilaisabel

[datetime.datetime(2015, 2, 8, 7, 9, 43),
 datetime.datetime(2015, 2, 8, 15, 9, 30),
 datetime.datetime(2015, 2, 8, 17, 50, 49),
 datetime.datetime(2015, 2, 8, 20, 28, 57)]

In [30]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [31]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            if(i+1 == len(departure_array_A)):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
         
            elif ((i+1 < len(departure_array_A)) and (departure_array_B[j] < departure_array_A[i+1])):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else: 
                i = i +1
            
        elif(departure_array_B[j] < departure_array_A[i]):
            if (j+1 == len(departure_array_B)):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
            elif ((j+1 < len(departure_array_B)) and (departure_array_A[i] < departure_array_B[j+1])):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else:
                j = j +1                
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  



In [32]:
#Funcion que mete todas las observaciones de GPS de un determinado bus_id en un vector
def build_vector_timestamp_latidude_longitude(bus_id): 
    vector = []
    for row in db.dados20150208_line_438.find({"bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        vector.insert(len(vector), [row[u'timestamp'], row[u'latitude'], row[u'longitude']])
    return vector

In [54]:
vector = build_vector_timestamp_latidude_longitude('A27629')
df_A27629 = pd.DataFrame(vector, columns=['timestamp', 'latitude','longitude'])       


In [37]:
#Funcion para calcular la distancia entre 2 puntos, teniendo en cuenta el radio de la tierra
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return (c * r)*1000 # return the distance in m

In [35]:
#funcion que dado un vector de lat, long y un punto A(lat,long), me devuelve un vector con la distancia de cada par al punto A

def vector_distancia(vector, point):
    vector_distance = []
    lat2 = point[0]
    long2 = point[1]
    for item in vector:
        lat1 = item[1]
        long1 = item[2]
        distance = haversine(long2, lat2, long1, lat1)
        latlon = (item[1], item[2])
        vector_distance.insert(len(vector_distance), [item[0],latlon, distance])
    df_vector_distance = pd.DataFrame(vector_distance, columns=['timestamp', 'latlon', 'distance'])
    return df_vector_distance   

In [41]:
dist_to_ini = vector_distancia(vector, initial_point)
sorted_dist_to_ini = dist_to_ini.sort(["timestamp"])


dist_to_fin = vector_distancia(vector, final_point)
sorted_dist_to_fin = dist_to_fin.sort(["timestamp"])


In [52]:
#hacer una grafica interactiva con la libreria de python (plotly.plotly). Se instala por consola asi: sudo pip install plotly
#distancia en cada horario del dia del punto inicial (Leblon)
#pegado a cero por el eje x va a estar mas proximo de leblon y el punto mas distante va a ser Vila Isabel
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_orange_stock.csv')
#df.head()

trace = Scatter( x=sorted_dist_to_ini['timestamp'], y = sorted_dist_to_ini['distance'] )
data = Data([trace])

py.iplot(data, filename='pandas-time-series')

In [51]:
#dados para llamar a la funcion

travel_time_table = create_travel_table("Leblon", arrive_hour_to_leblon, leblon_departure_hours, "Vila Isabel", arrive_hour_to_vilaisabel, vilaIsabel_departure_hours, 'A27629', '438' )
travel_time_table

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27629,Vila Isabel_to_Leblon,2015-02-08 18:23:57,2015-02-08 19:23:27,0:59
1,438,A27629,Leblon_to_Vila Isabel,2015-02-08 19:23:27,2015-02-08 20:28:57,1:05


In [70]:
df_A27629_query = df_A27629[(df_A27629["timestamp"]== '08-02-2015 05:30:00') ]


In [73]:
for row in df_A27629_query.iterrows():
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap_A27629.polygon_marker( latlon, fill_color='cean', num_sides=4, radius=6)

embed_map(fmap_A27629)

In [ ]:
6:10 - 6:20
7:40-7:50
8:20-8:30
14:00 - 14:10
14:30 - 14:40
16:20 - 16:30
        17:10-17:20
                19:20




In [51]:
sorted_travel_time_table_A27611 = mytabla1.sort(["Departure_Hour"])
sorted_travel_time_table_A27611

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
